In [3]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install catboost==0.26.1
!pip install xgboost==1.4.2
import re
import time
import copy
import pickle
import collections
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

# 파일로딩

In [4]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.
old_data = pd.read_excel('공모전 제공 데이터(1차).xlsx')
data = pd.read_excel('공모전 제공 데이터(2차).xlsx')

In [6]:
# 복문 나눠 데이터 개수 증가

data = pd.concat([data,old_data], ignore_index=True)
X = data['발화']

label = []
bokmoon = []
feature = []

oneindex = 0
count = 0
for i in range(len(X)):
    if (len(str(data['발화2'][i])) != 3): ## 복문일때
        bokmoon.append(X[i])
        count = count + 1
        
        if (X[i].find('니다') != -1 and len(X[i][X[i].find('니다')+2:]) > 7): ## 발화 1 처리
            feature.append(X[i][:X[i].find('니다')+2])
            label.append(data['발화1'][i])
            oneindex = X[i].find('니다')+2
        elif X[i].find('고 ') != -1 and len(X[i][X[i].find('고 ')+2:]) > 7:
            feature.append(X[i][:X[i].find('고 ')+2])
            label.append(data['발화1'][i])
            oneindex = X[i].find('고 ')+2     
        elif (X[i].find('서 ') != -1 and len(X[i][X[i].find('서 ')+2:]) > 7):
            feature.append(X[i][:X[i].find('서 ')+2])
            label.append(data['발화1'][i])
            oneindex = X[i].find('서 ')+2
        elif (X[i].find('다 ') != -1 and len(X[i][X[i].find('다 ')+2:]) > 7):
            feature.append(X[i][:X[i].find('다 ')+2])
            label.append(data['발화1'][i])
            oneindex = X[i].find('다 ')+2
        elif (X[i].find('하고') != -1 and len(X[i][X[i].find('하고')+2:]) > 7):
            feature.append(X[i][:X[i].find('하고')+2])
            label.append(data['발화1'][i])
            oneindex = X[i].find('하고')+2
        else:
            feature.append(X[i][:(int(len(X[i])/2))+1])
            label.append(data['발화1'][i])
            oneindex = int(len(X[i])/2)+1

        feature.append(X[i][oneindex:])
        label.append(data['발화2'][i])
    else:
        feature.append(X[i])
        label.append(data['최종분류(우선순위 가장 높은것 선택)'][i])
        
X = feature 
label = pd.DataFrame(label)

# X의 불필요 요소 제거
for i in range(len(X)):
    X[i] = X[i].replace(' ', '');    X[i] = X[i].replace(',', '');    X[i] = X[i].replace('.', '');    X[i] = X[i].replace(';', '');    X[i] = X[i].replace('~', '');    X[i] = X[i].replace('❤', '');    X[i] = X[i].replace('ㆍ', '');    X[i] = X[i].replace('_', '');    X[i] = X[i].replace('-', '');    X[i] = X[i].replace('(', '');    X[i] = X[i].replace(')', '');    X[i] = X[i].replace(':', '');    X[i] = X[i].replace('?', '');    X[i] = X[i].replace('!', ''); X[i] = X[i].replace('ᆢ', '')


In [7]:
def typo_detection(X):
  sec_syllable = []
  for i in X:
      for j in range(len(i)):
          if (j+1 >= len(i)):
              break
          sec_syllable.append(i[j] + i[j+1])
          
  sec_tokenizer = Tokenizer()
  sec_tokenizer.fit_on_texts(sec_syllable) # 10000개의 행을 가진 X의 각 행에 토큰화를 수행
  sec_sequences = sec_tokenizer.texts_to_sequences(X) # 단어를 숫자값, 인덱스로 변환하여 저장

  sec_wti = sec_tokenizer.word_index
  search = []

  count = 0
  for i in sec_wti:
      search.append(i)
      count += 1
      
  def sort_by_frequency(data):
      rt_data = []
      for d, c in Counter(data).most_common():
          for i in range(c):
              rt_data.append(d)
      return rt_data

  sort = sort_by_frequency(sec_syllable)


  # 단어를 자음 모음으로 분류
  class SeparateJaMo:
      def __init__(self, original_text, blank_str=" ",
                  remove_blank=False, remove_special_character=False,
                  refine_blank=False, refine_english=False):
          self.__chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
          self.__jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
          self.__jongsung_list = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
          self.__original_text = original_text
          self.__blank_str = blank_str
          self.__remove_blank = remove_blank
          self.__remove_special_character = remove_special_character
          self.__refine_blank = refine_blank
          self.__refine_english = refine_english
          self.__processed_text = self.__make_processed_text()
          self.__jamo_list = self.__make_jamo_list()

      def __make_processed_text(self):
          # 옵션에 맞게 문자열을 정제하여 반환
          processed_text = self.__original_text
          if self.__refine_blank:
              processed_text = " ".join(
                  [word.strip() for word in processed_text.split(" ") if not len(word.strip()) == 0]
              )
          if self.__remove_blank:
              processed_text = re.sub(r'\s', '', processed_text)
          if self.__remove_special_character:
              processed_text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', processed_text)
              processed_text = re.sub(r'\W \S', '', processed_text)
          if self.__refine_english:
              processed_text = re.sub(r'[^ㄱ-ㅎㅣ가-힣]+', '', processed_text)
          processed_text = re.sub(r'\s', self.__blank_str, processed_text)
          return processed_text

      def __make_jamo_list(self):
          # 전체 자모 리스트 반환
          jamo_list = list()
          for syllable in list(self.__processed_text):
              if re.match(r'[ㄱ-ㅎㅣ가-힣]+', syllable):
                  syllable_code = ord(syllable)
                  if syllable_code < 12700:
                      jamo_list.append([syllable,'', ''])
                  else:
                      chosung_index = int((syllable_code - 44032) / 588)
                      jungsung_index = int((syllable_code - 44032 - (chosung_index * 588)) / 28)
                      jongsung_index = int(syllable_code - 44032 - (chosung_index * 588) - (jungsung_index * 28))
                      jamo_list.append(
                          [
                              self.__chosung_list[chosung_index],
                              self.__jungsung_list[jungsung_index],
                              self.__jongsung_list[jongsung_index]
                          ]
                      )
              else:
                  jamo_list.append([syllable, syllable, syllable])
          return jamo_list

      def get_full_jamo_list(self):
          return self.__jamo_list
      
  # search 단어 자모로 쪼개기
  separate_jamo_full = []

  for i in range(len(sort)):
      if sec_syllable[i][1] != '':
          text = sort[i]
          separate_jamo = SeparateJaMo(text, refine_blank=True, remove_special_character=True)
          separate_jamo_full.append(separate_jamo.get_full_jamo_list())

  # search 안 단어의 좌표 설정 (separate_jamo_full)
  address = [[0 for col in range(6)] for row in range(len(separate_jamo_full))] ## 좌표를 저장할 공간

  for k in range(len(separate_jamo_full)):
      for e in range(len(separate_jamo_full[k])): # 음절 수
          for j in range(len(separate_jamo_full[k][e])): #  각 음절의 자음모음수
              # 2 3 음절의 좌표 저장 오류 고치기
              if (separate_jamo_full[k][e][j] == 'ㅂ' or separate_jamo_full[k][e][j] == 'ㅃ' or separate_jamo_full[k][e][j] == 'ㅐ' or separate_jamo_full[k][e][j] == 'ㅔ'): # 9
                  if (e == 0):
                      address[k][j] = 14
                  elif (e == 1):
                      address[k][j+3] = 14
                  elif (e == 2):
                      address[k][j+6] = 14
              elif (separate_jamo_full[k][e][j] == 'ㅍ'): # 9.2
                  if (e == 0):
                      address[k][j] = 15
                  elif (e == 1):
                      address[k][j+3] = 15
                  elif (e == 2):
                      address[k][j+6] = 15
              elif (separate_jamo_full[k][e][j] == 'ㅈ' or separate_jamo_full[k][e][j] == 'ㅉ' or separate_jamo_full[k][e][j] == 'ㅑ'): # 8.5
                  if (e == 0):
                      address[k][j] = 13
                  elif (e == 1):
                      address[k][j+3] = 13
                  elif (e == 2):
                      address[k][j+6] = 13
              elif (separate_jamo_full[k][e][j] == 'ㅊ'): # 8.2
                  if (e == 0):
                      address[k][j] = 12
                  elif (e == 1):
                      address[k][j+3] = 12
                  elif (e == 2):
                      address[k][j+6] = 12
              elif (separate_jamo_full[k][e][j] == 'ㅣ'): # 8
                  if (e == 0):
                      address[k][j] = 11
                  elif (e == 1):
                      address[k][j+3] = 11
                  elif (e == 2):
                      address[k][j+6] = 11
              elif (separate_jamo_full[k][e][j] == 'ㅁ'): # 7.5
                  if (e == 0):
                      address[k][j] = 10
                  elif (e == 1):
                      address[k][j+3] = 10
                  elif (e == 2):
                      address[k][j+6] = 10
              elif (separate_jamo_full[k][e][j] == 'ㄴ' or separate_jamo_full[k][e][j] == 'ㅏ'): # 7
                  if (e == 0):
                      address[k][j] = 9
                  elif (e ==1):
                      address[k][j+3] = 9
                  elif (e == 2):
                      address[k][j+6] = 9
              elif (separate_jamo_full[k][e][j] == 'ㄹ' or separate_jamo_full[k][e][j] == 'ㅓ'): # 6.5
                  if (e == 0):
                      address[k][j] = 8
                  elif (e ==1):
                      address[k][j+3] = 8
                  elif (e == 2):
                      address[k][j+6] = 8
              elif (separate_jamo_full[k][e][j] == 'ㄷ' or separate_jamo_full[k][e][j] == 'ㄸ'): # 6
                  if (e == 0):
                      address[k][j] = 7
                  elif (e == 1):
                      address[k][j+3] = 7
                  elif (e == 2):
                      address[k][j+6] = 7
              elif (separate_jamo_full[k][e][j] == 'ㅌ'): # 5.7
                  if (e == 0):
                      address[k][j] = 6.5
                  elif (e == 1):
                      address[k][j+3] = 6.5
                  elif (e == 2):
                      address[k][j+6] = 6.5
              elif (separate_jamo_full[k][e][j] == 'ㅇ'): # 5.5
                  if (e == 0):
                      address[k][j] = 6
                  elif (e == 1):
                      address[k][j+3] = 6
                  elif (e == 2):
                      address[k][j+6] = 6
              elif (separate_jamo_full[k][e][j] == 'ㅋ'): # 5.2
                  if (e == 0):
                      address[k][j] = 5.5
                  elif (e == 1):
                      address[k][j+3] = 5.5
                  elif (e == 2):
                      address[k][j+6] =5.5
              elif (separate_jamo_full[k][e][j] == 'ㄱ' or separate_jamo_full[k][e][j] == 'ㄲ' or separate_jamo_full[k][e][j] == 'ㅎ'): # 5
                  if (e == 0):
                      address[k][j] = 5
                  elif (e == 1):
                      address[k][j+3] = 5
                  elif (e == 2):
                      address[k][j+6] = 5
              elif (separate_jamo_full[k][e][j] == 'ㅗ' or separate_jamo_full[k][e][j] == 'ㅜ'): # 4.5
                  if (e == 0):
                      address[k][j] = 4
                  elif (e == 1):
                      address[k][j+3] = 4
                  elif (e == 2):
                      address[k][j+6] = 4
              elif (separate_jamo_full[k][e][j] == 'ㅡ' or separate_jamo_full[k][e][j] == 'ㅕ'): # 4
                  if (e == 0):
                      address[k][j] = 3
                  elif (e == 1):
                      address[k][j+3] = 3
                  elif (e == 2):
                      address[k][j+6] = 3
              elif (separate_jamo_full[k][e][j] == 'ㅛ'): # 3.5
                  if (e == 0):
                      address[k][j] = 2
                  elif (e == 1):
                      address[k][j+3] = 2
                  elif (e == 2):
                      address[k][j+6] = 2
              elif (separate_jamo_full[k][e][j] == 'ㅅ' or separate_jamo_full[k][e][j] == 'ㅆ'): # 3
                  if (e == 0):
                      address[k][j] = 1
                  elif (e == 1):
                      address[k][j+3] = 1
                  elif (e == 2):
                      address[k][j+6] = 1
              elif (separate_jamo_full[k][e][j] == ''): # -1
                  if (e == 0):
                      address[k][j] = -1
                  elif (e == 1):
                      address[k][j+3] = -1
                  elif (e == 2):
                      address[k][j+6] = -1
              elif (separate_jamo_full[k][e][j] == 'ㄳ' or separate_jamo_full[k][e][j] == 'ㄵ' or separate_jamo_full[k][e][j] =='ㄶ'
                  or separate_jamo_full[k][e][j] =='ㄺ' or separate_jamo_full[k][e][j] =='ㄻ' or separate_jamo_full[k][e][j] =='ㄼ'
                  or separate_jamo_full[k][e][j] == 'ㄽ' or separate_jamo_full[k][e][j] =='ㄾ' or separate_jamo_full[k][e][j] =='ㄿ'
                  or separate_jamo_full[k][e][j] =='ㅀ' or separate_jamo_full[k][e][j] =='ㅄ'):
                  if (e == 0):
                      address[k][j] = 8
                  elif (e == 1):
                      address[k][j+3] = 8
                  elif (e == 2):
                      address[k][j+6] = 8
              else:
                  if (e == 0):
                      address[k][j] = 8
                  elif (e == 1):
                      address[k][j+3] = 8
                  elif (e == 2):
                      address[k][j+6] = 8

  ## knn으로 빈도수 높은 2음절 찾기
  knn_model = KNeighborsClassifier(n_neighbors=3)
  knn_model.fit(address, sort)
  pred = knn_model.predict(address)
  ## sorting
  pred_sort = sort_by_frequency(pred)

  pred_list = []
  for k in range(len(pred)):
      pred_list.append(pred[k])

  sort_set = []
  pred_set = []
  for i, j in zip(sort,pred_list):
      if (i not in sort_set):
          sort_set.append(i)
          pred_set.append(j)

  ## sort : origin y, pred : pred y, pred_sort, pred_list : pred sort y
  for i in range(len(sort_set)): ## 단어들
      if (47 < ord(sort_set[i][0]) < 123):
          continue
      if (sort_set[i] != pred_set[i] and i != count+1 and pred_list.count(pred_set[i]) > sort.count(sort_set[i]) * 10 and 10 < sort.count(sort_set[i]) < 30):
          for j in range(len(X)): # 문장들
              if (X[j].find(sort_set[i]) != -1): # 문장에 바꾸고자하는 단어가 있다면
                  for k in range(len(X[j])-3): # 문장속
                      if (X[j][k:k+2] == sort_set[i]):
                          if (sort.count(X[j][k+1:k+3]) < sort.count(X[j][k:k+2])):
                              X[j] = X[j].replace(X[j][k:k+2], pred_set[i])
                              count = i
  return X

In [8]:
X = typo_detection(X) # 오타 수정

bin_label = copy.deepcopy(label)
cat_label = copy.deepcopy(label)
for i in range(len(label)):
    bin_label[0][i] = bin_label[0][i][0:2]
    cat_label[0][i] = cat_label[0][i][3:]
    if cat_label[0][i] == '':
       cat_label[0][i] = '없음'

y_bin = bin_label
y_cat = cat_label


# s_count에 따라 몇 개를 할 지 판단
s_count = 550

# 2음절은 s_count의 3/4만큼을 차지함
find_all = []
sec_syllable = []
for i in X:
    for j in range(len(i)):
        if (j+1 >= len(i)):
            break
        sec_syllable.append(i[j] + i[j+1])
sec_tokenizer = Tokenizer()
sec_tokenizer.fit_on_texts(sec_syllable) # 1000개의 행을 가진 X의 각 행에 토큰화를 수행
sec_sequences = sec_tokenizer.texts_to_sequences(X) # 단어를 숫자값, 인덱스로 변환하여 저장

sec_wti = sec_tokenizer.word_index

count = 0
for i in sec_wti:
    if len(i) < 2:
        continue
    if count == int(s_count/4*3): 
        break
    find_all.append(i)
    count += 1

# 3음절은 s_count의 1/4만큼을 차지함
trd_syllable = []
for i in X:
    for j in range(len(i)):
        if (j+2 >= len(i)):
            break
        trd_syllable.append(i[j] + i[j+1] + i[j+2])

trd_tokenizer = Tokenizer()
trd_tokenizer.fit_on_texts(trd_syllable) # 1000개의 행을 가진 X의 각 행에 토큰화를 수행
trd_sequences = trd_tokenizer.texts_to_sequences(X) # 단어를 숫자값, 인덱스로 변환하여 저장
trd_wti = trd_tokenizer.word_index

count = 0
for i in trd_wti:
    if len(i) < 3:
        continue
    if count == int(s_count/4):
        break
    find_all.append(i)
    count += 1

# y를 포함한 feature을 세팅하는 장소
feature = pd.DataFrame(index = X, columns=find_all)
for i in range(len(feature.index)):
    for k in range(len(find_all)):
        if (feature.index[i].find(find_all[k]) != -1):
            feature.iloc[i][k] = 1
        else:
            feature.iloc[i][k] = 0

In [9]:
y_cat_one = []
y_cat_two = []

y_bin = y_bin[0].to_list()
y_cat = y_cat[0].to_list()

for i in range(len(y_cat)):
    if y_cat[i].find('>') == -1:
        y_cat_one.append(y_cat[i])
        y_cat_two.append('없음')
    else:
        y_cat_one.append(y_cat[i][:y_cat[i].find('>')])
        y_cat_two.append(y_cat[i][y_cat[i].find('>'):])

le_bin = LabelEncoder()
le_cat_one = LabelEncoder()
le_cat_two = LabelEncoder()
label_list_bin = []
label_list_cat_one = []
label_list_cat_two = []

for i in y_bin:
    if i not in label_list_bin:
        label_list_bin.append(i)
for i in y_cat_one:
    if i not in label_list_cat_one:
        label_list_cat_one.append(i)    
for i in y_cat_two:
    if i not in label_list_cat_two:
        label_list_cat_two.append(i)
        
le_bin.fit(label_list_bin)
le_cat_one.fit(label_list_cat_one)
le_cat_two.fit(label_list_cat_two)

y_bin = le_bin.transform(y_bin)
y_cat_one = le_cat_one.transform(y_cat_one)
y_cat_two = le_cat_two.transform(y_cat_two)

In [10]:
def feature_selection(y):
    # n_count로 2음절, 3음절 합의 개수 선택
    n_count = 535
    # RandomForestRegressor로 중요도 낮은 feature 걸러내기 
    n_feature = find_all
    index = np.arange(len(n_feature))

    rfc = RandomForestClassifier(random_state=1228)
    rfc.fit(feature, y)

    allfeatRF = np.empty((0,2))
    for i in range(len(rfc.feature_importances_)):
        allfeatRF = np.append(allfeatRF, np.array([[rfc.feature_importances_[i], n_feature[i]]]), axis=0)

    allfeatRF = allfeatRF[allfeatRF[:,0].argsort()]

    featRF = np.empty((0,2))
    for i in range(len(allfeatRF)-n_count, len(allfeatRF)):
        featRF = np.append(featRF, np.array([[allfeatRF[i][0], allfeatRF[i][1]]]), axis=0)

    # LightGBMClassifier 중요도 낮은 feature 걸러내기 
    lgbc = lgb.LGBMClassifier(random_state=1228)
    lgbc.fit(feature, y)

    allfeatLGBC = np.empty((0,2))
    for i in range(len(lgbc.feature_importances_)):
        allfeatLGBC = np.append(allfeatLGBC, np.array([[lgbc.feature_importances_[i], n_feature[i]]]), axis=0)

    allfeatLGBC = allfeatLGBC[allfeatLGBC[:,0].argsort()]

    featLGBC = np.empty((0,2))
    for i in range(len(allfeatLGBC)-n_count, len(allfeatLGBC)):
        featLGBC = np.append(featLGBC, np.array([[allfeatLGBC[i][0], allfeatLGBC[i][1]]]), axis=0)

    # LightGBMClassifier와 RandomForestClassifier의 중요도 선별 후 중복되는 feature 선정
    featUse = np.empty((0,2))
    for i in featLGBC:
        for j in featRF:
            if (i[1] == j[1]):
                featUse = np.append(featUse, np.array([[i[0], i[1]]]), axis=0)

    sel_feat = []
    sel_col = []
    for i in range(len(featUse)):
        if 'e-' not in featUse[i][0]:
            sel_feat.append(featUse[i])

    for i in sel_feat:
        sel_col.append(i[1])
    
    return sel_col


def new_feature_creation(y):
    feature_new = pd.DataFrame(index = feature.index, columns=y)
    for i in y:
        for j in feature.columns:
            if i == j:
                feature_new[i] = feature[j]
                
    return feature_new

for i in find_all:
    feature[i] = pd.to_numeric(feature[i])

sel_col_bin = feature_selection(y_bin)
sel_col_cat_one = feature_selection(y_cat_one)
sel_col_cat_two = feature_selection(y_cat_two)

feature_new_bin = new_feature_creation(sel_col_bin)
feature_new_cat_one = new_feature_creation(sel_col_cat_one)
feature_new_cat_two = new_feature_creation(sel_col_cat_two)